In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg,col,first,when,udf
from pyspark.sql.types import *
ss = SparkSession.builder.getOrCreate()

In [22]:
df = ss.read.format("csv").options(header='true', delimiter=',').load("gruposT.csv")

In [23]:
df.show()

+-----+-----------------+-----+--------------------+------+
|grupo|       clientes_t|contT|         clientes_t6|contT6|
+-----+-----------------+-----+--------------------+------+
| asdf|      1,2,3,4,5,6|    6|1,2,3,4,5,6,7,8,9,10|    10|
| qwer|   11,12,13,14,15|    5|                   0|     0|
| zxcv|28,29,30,31,32,33|    6|20,21,21,23,24,25...|     8|
+-----+-----------------+-----+--------------------+------+



In [24]:
def cortarCadena(cadena):
    return cadena.split(",")

In [25]:
def permanencia(clientesT, clentesT6):
    return len([x for x in clientesT if x in clentesT6])

In [33]:
#params(NoPermanencia, ConteoMesT-6)
def validarPerm(perm, lenT6):
    perm = int(perm)
    lenT6 = int(lenT6)
    if lenT6 == 0:#Caso en que el grupo es nuevo Ejemplo 2
        return 2.0
    elif lenT6 > 0 and perm == 0:#Caso en que los que dispersaron en marzo no dispersaron en sep Ejemplo 3
        return 0.0
    elif lenT6 > 0 and perm > 0:#Caso en que se tienen algunos que dispersaron septiembre y en marzo Ejemplo 1
        return perm/lenT6

In [34]:
def getRotacion(valPermanencia):
    return 1 - valPermanencia

In [40]:
cortarCadena_udf = udf(cortarCadena, ArrayType(StringType()))
permanencia_udf = udf(permanencia, IntegerType())
validarPerm_udf = udf(validarPerm, FloatType())
getRotacion_udf = udf(getRotacion, FloatType())

In [41]:
df = df.withColumn("arrClientesT", cortarCadena_udf(df.clientes_t))
df = df.withColumn("arrClientesT6", cortarCadena_udf(df.clientes_t6))

In [42]:
df = df.withColumn("permanencia", permanencia_udf(df.arrClientesT, df.arrClientesT6))

In [43]:
df = df.withColumn("valPermanencia", validarPerm_udf(df.permanencia, df.contT6))

In [44]:
df = df.withColumn("valRotacion", getRotacion_udf(df.valPermanencia))
df.show()

+-----+-----------------+-----+--------------------+------+--------------------+--------------------+-----------+--------------+-----------+
|grupo|       clientes_t|contT|         clientes_t6|contT6|        arrClientesT|       arrClientesT6|permanencia|valPermanencia|valRotacion|
+-----+-----------------+-----+--------------------+------+--------------------+--------------------+-----------+--------------+-----------+
| asdf|      1,2,3,4,5,6|    6|1,2,3,4,5,6,7,8,9,10|    10|  [1, 2, 3, 4, 5, 6]|[1, 2, 3, 4, 5, 6...|          6|           0.6|        0.4|
| qwer|   11,12,13,14,15|    5|                   0|     0|[11, 12, 13, 14, 15]|                 [0]|          0|           2.0|       -1.0|
| zxcv|28,29,30,31,32,33|    6|20,21,21,23,24,25...|     8|[28, 29, 30, 31, ...|[20, 21, 21, 23, ...|          0|           0.0|        1.0|
+-----+-----------------+-----+--------------------+------+--------------------+--------------------+-----------+--------------+-----------+

